In [14]:
from googlesearch import search 
import import_ipynb
from bs4 import BeautifulSoup as bsoup
from multiprocessing.dummy import Pool
import requests
import sys
from requests.exceptions import RequestException
from contextlib import closing
import re
import json
import os

In [2]:
def content_Preprocess(content):
    if content is not None:
        title=''
        text=''
        soup = bsoup(content, 'html.parser')
        try:
            title += soup.find('title').getText()
        except AttributeError as error:
            print(error)
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        # get text
        text += soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        return (title, text)
    else:
        return None

In [3]:
def simple_get(url):
    try:
        with closing(requests.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    print(e)

In [4]:
def load_json(filename, directory='./tmp/'):
    path=os.path.join(directory, filename + "." + 'json')
    with open(path) as f:
        data = json.load(f)
    return data

In [15]:
def get_colleage(name):
    ent_name=name.replace(' +',"_")
    content=simple_get('http://en.wikipedia.org/wiki/'+ent_name)
    if content is not None:
        soup = bsoup(content, 'html.parser')
        table = soup.find('table', attrs={'class':'infobox biography vcard'})
        if table is not None:
            table_body = table.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                col_hd = row.find('th')
                col=row.find('td')
                if col_hd is not None and col is not None:
                    col = col.text.strip()
                    col_hd=col_hd.text.strip()
                    if "Alma" in col_hd:
                        return (name,col.split("\n")[-1])

In [191]:
def multiple_process(ent_name, get_colleage, proc=2):
    p=Pool(proc)
    try:
        cllg=p.map(get_colleage, ent_name)
    except Exception as e:
        print(e)
    return cllg

In [ ]:
ent_name=[]
pid_name_entid=load_json('pid2ent2','./')
for a in pid_name_entid:
    ent_name.append(a[1])
a=multiple_process(ent_name, get_colleage)

In [ ]:
"^(?=.*?\b[A|a]lan\b)(?=.*?\b[T|t]uring\b)(?=.*?\b[D|d]octorate\b).*$"

In [ ]:
query = 'Alan turing AND doctorate institute'
repeat=[]
for url in search(query, stop=20):
    if 'YouTube' not in url:
        if not any((a in url or url in a) for a in repeat):
            r=simple_get(url)
            if content_Preprocess(r) is not None:
                title_text=content_Preprocess(r)
                print(url)
                match=re.search("([Aa]lan|[Tt]uring).*([Dd]octorate).*?\.",title_text[1])
                if match is not None:
                    print(match.group())
                repeat.append(url)
            else:
                continue
        else:
            continue